In [1]:
print("hello")

hello


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import matplotlib.pyplot as plt

# 1. モデルとトークナイザーの準備 (Qwen-7B-Chat or Base)
model_name = "Qwen/Qwen2.5-7B-Instruct" # またはお手持ちのパス
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype="auto", 
    device_map="auto"
)

def calculate_entropy_and_aha_score(text, window_size=5):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits  # [1, seq_len, vocab_size]
        
    # 各ステップの予測分布からエントロピーを計算
    probs = torch.softmax(logits[:, :-1, :], dim=-1)
    log_probs = torch.log_softmax(logits[:, :-1, :], dim=-1)
    token_entropies = -torch.sum(probs * log_probs, dim=-1).squeeze(0).float().cpu().numpy()
    
    # Aha! Scoreの計算: (過去Nトークンの最大エントロピー) - (現在のエントロピー)
    aha_scores = []
    for i in range(len(token_entropies)):
        if i < window_size:
            aha_scores.append(0)
            continue
        
        past_window = token_entropies[i-window_size : i]
        max_past_entropy = np.max(past_window)
        current_entropy = token_entropies[i]
        
        # 過去に迷い（高エントロピー）があり、今スッキリ（低エントロピー）した度合い
        score = max(0, max_past_entropy - current_entropy)
        aha_scores.append(score)
        
    return token_entropies, aha_scores, tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][1:])

# 2. 検証する3つのシナリオ
scenarios = {
    "Aha! (Discovery)": "Wait, if we apply quantum superposition to apple sorting, we can observe all states simultaneously. Yes, that's the key to efficiency!",
    "Chaos (Nonsense)": "Apple is a quantum blue banana flying through the square root of flavor gravity and orange noise.",
    "Boring (Commonplace)": "An apple is a common fruit that grows on trees and is often red or green in color."
}

# 3. 可視化と評価
plt.figure(figsize=(12, 8))

for name, text in scenarios.items():
    entropies, aha_scores, tokens = calculate_entropy_and_aha_score(text)
    
    print(f"\n--- Scenario: {name} ---")
    print(f"Total Aha! Score: {sum(aha_scores):.4f}")
    print(f"Max Aha! Peak: {max(aha_scores):.4f}")

    plt.plot(aha_scores, label=f"{name} (Total: {sum(aha_scores):.2f})")

plt.title("Aha! Reward Dynamics Comparison")
plt.xlabel("Token Position")
plt.ylabel("Aha! Score (Entropy Drop)")
plt.legend()
plt.grid(True)
plt.show()

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

TypeError: Got unsupported ScalarType BFloat16

<Figure size 1200x800 with 0 Axes>